<a href="https://colab.research.google.com/github/signalcompose/MusicTransformer-tensorflow2.0/blob/master/GoogleCoraboratory/MusicTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Google Drive をマウント

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### マウントされたドライブに移動
#### GoogleColaboratory ディレクトリにリポジトリをクローンしていない場合
* GoogleColaboratory ディレクトリに移動
* MusicTransformerを git clone する
* リポジトリのサブモジュールをダウンロード
* [将来削除予定]経過時間を表示するブランチにチェックアウトする

/content


In [12]:
%cd drive/'My Drive'/GoogleColaboratory
!git clone https://github.com/signalcompose/MusicTransformer-tensorflow2.0.git 
%cd MusicTransformer-tensorflow2.0/
!git submodule init && git submodule update
!git checkout -b feature/PrintTrainTimer origin/feature/PrintTrainTimer

/content/drive/My Drive/GoogleColaboratory
Cloning into 'MusicTransformer-tensorflow2.0'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 3109 (delta 36), reused 34 (delta 18), pack-reused 3036
Receiving objects: 100% (3109/3109), 56.45 MiB | 9.80 MiB/s, done.
Resolving deltas: 100% (298/298), done.
/content/drive/My Drive/GoogleColaboratory/MusicTransformer-tensorflow2.0
Submodule 'midi_processor' (https://github.com/jason9693/midi-neural-processor.git) registered for path 'midi_processor'
Cloning into '/content/drive/My Drive/GoogleColaboratory/MusicTransformer-tensorflow2.0/midi_processor'...
Submodule path 'midi_processor': checked out 'bea0dc612b7f687f964d0f6d54d1dbf117ae1307'
Branch 'feature/PrintTrainTimer' set up to track remote branch 'feature/PrintTrainTimer' from 'origin'.
Switched to a new branch 'feature/PrintTrainTimer'


#### 既に MusicTransformer を git clone している場合

In [2]:
%cd drive/'My Drive'/GoogleColaboratory/MusicTransformer-tensorflow2.0


/content/drive/My Drive/GoogleColaboratory/MusicTransformer-tensorflow2.0


### 必要なモジュールのインストール


In [3]:
!pip install progress pretty_midi config

     |████████████████████████████████| 5.6MB 5.4MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
  Created wheel for progress: filename=progress-1.5-cp36-none-any.whl size=8074 sha256=7507e6a31dd924e191307695133cd492859712f8352e76987e43c91010ba2688
  Stored in directory: /root/.cache/pip/wheels/6c/c8/80/32a294e3041f006c661838c05a411c7b7ffc60ff939d14e116
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-cp36-none-any.whl size=5591953 sha256=1d20427e515c112c4301b3eb934be8eb9a7091c2548cbcaf215f15a86ccd41ea
  Stored in directory: /root/.cache/pip/wheels/4c/a1/c6/b5697841db1112c6e5866d75a6b6bf1bef73b874782556ba66
Successfully built progress pretty-midi


### データのダウンロード
* 保存先のディレクトリ作成
* Yamaha e-piano competition のデータダウンロード

In [14]:
!mkdir -p dataset/ecomp/midi dataset/ecomp/processed dataset/ecomp/generated
!sh dataset/scripts/ecomp_piano_downloader.sh dataset/ecomp/midi

ストリーミング出力は最後の 5000 行に切り捨てられました。

2020-06-04 13:29:13 (18.2 MB/s) - ‘dataset/ecomp/midi/Levitsky02.MID’ saved [55735/55735]

--2020-06-04 13:29:13--  http://www.piano-e-competition.com/MIDIFiles/2008/Levitsky03.MID
Reusing existing connection to www.piano-e-competition.com:80.
HTTP request sent, awaiting response... 200 OK
Length: 85986 (84K) [audio/mid]
Saving to: ‘dataset/ecomp/midi/Levitsky03.MID’

Levitsky03.MID      100%[===================>]  83.97K  --.-KB/s    in 0.001s  

2020-06-04 13:29:13 (55.7 MB/s) - ‘dataset/ecomp/midi/Levitsky03.MID’ saved [85986/85986]

--2020-06-04 13:29:13--  http://www.piano-e-competition.com/MIDIFiles/2008/Levitsky04.MID
Reusing existing connection to www.piano-e-competition.com:80.
HTTP request sent, awaiting response... 200 OK
Length: 20506 (20K) [audio/mid]
Saving to: ‘dataset/ecomp/midi/Levitsky04.MID’

Levitsky04.MID      100%[===================>]  20.03K  --.-KB/s    in 0.001s  

2020-06-04 13:29:13 (19.1 MB/s) - ‘dataset/ecomp/midi/Levitsky0

### preprocess.py の実行


In [15]:
!python preprocess.py dataset/ecomp/midi dataset/ecomp/processed

2020-06-04 13:29:48.485827: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Processing |################################| 1374/1374


## トレーニング


In [0]:
!python train.py --epochs=200 --batch_size=2 --save_path=dataset/ecomp/model --pickle_dir=dataset/ecomp/processed

2020-06-04 13:47:21.226496: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
<class Data has "1374" files>
2020-06-04 13:47:23.705518: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-06-04 13:47:23.720378: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-06-04 13:47:23.721225: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-06-04 13:47:23.721298: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-06-04 13:47:23.723002: I

## Midiファイル出力

In [0]:
!python generate.py --load_path=dataset/ecomp/model --prior_midi=dataset/ecomp/midi/BENABD10.mid --save_path=dataset/ecomp/generated/test.mid --length=1000